# Kenyan Sign Language Recognition with TensorFlow Lite

This notebook demonstrates how to create a TensorFlow Lite model for recognizing Kenyan Sign Language (KSL) and translating it to English text. We'll use the [KSL Dataset from Kaggle](https://www.kaggle.com/datasets/yonaschanie/kslc-dataset) for training our model.

## 1. Install and Import Required Libraries

First, let's install and import all the necessary libraries for our project.

In [ ]:
# Install required packages
!pip install tensorflow tensorflow-hub matplotlib numpy pandas opencv-python scikit-learn kagglehub pillow

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
from PIL import Image
import kagglehub

print("TensorFlow version:", tf.__version__)

## 2. Download and Explore the Dataset

We'll use the Kenyan Sign Language Classification (KSLC) dataset available on Kaggle.

In [ ]:
# Download latest version of the dataset
path = kagglehub.dataset_download("yonaschanie/kslc-dataset")

print("Path to dataset files:", path)

In [ ]:
# Explore the dataset structure
!ls -la {path}

### Load and preprocess the dataset

Let's create functions to load and preprocess the images from the dataset.

In [ ]:
# Define image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

In [ ]:
def load_and_preprocess_data(data_path):
    """
    Load and preprocess the Kenyan Sign Language dataset
    """
    # This function assumes the dataset is organized in folders by class
    # Adjust accordingly based on the actual structure of the dataset
    
    # Create ImageDataGenerator for data augmentation and normalization
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load training data
    train_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )
    
    # Load validation data
    validation_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )
    
    # Get class names and indices
    class_names = sorted(list(train_generator.class_indices.keys()))
    print(f"Found {len(class_names)} classes: {class_names}")
    
    return train_generator, validation_generator, class_names

In [ ]:
# Load and preprocess the data
try:
    # First, try to find the dataset structure
    !find {path} -type d | sort
    
    # Assuming the structure contains training data in a directory
    # Adjust the path according to the actual dataset structure
    dataset_path = path
    
    # Load the data
    train_generator, validation_generator, class_names = load_and_preprocess_data(dataset_path)
except Exception as e:
    print(f"Error loading the dataset: {e}")
    print("Please adjust the path to point to the folder containing class subfolders.")

## 3. Create and Train the Model

We'll use transfer learning with MobileNetV2, which is both powerful and lightweight - making it ideal for conversion to TensorFlow Lite later.

In [ ]:
def create_model(num_classes):
    """
    Create a transfer learning model based on MobileNetV2
    """
    # Load the MobileNetV2 model with pre-trained weights on ImageNet
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Create the model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
try:
    # Create and train the model
    num_classes = len(class_names)
    model = create_model(num_classes)
    
    # Display model summary
    model.summary()
    
    # Define callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
    ]
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=callbacks
    )
except Exception as e:
    print(f"Error training model: {e}")

### Visualize Training Results

In [ ]:
def plot_training_history(history):
    """
    Plot the training and validation accuracy/loss
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy plot
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='lower right')
    
    # Loss plot
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.show()

In [ ]:
try:
    # Plot training history
    plot_training_history(history)
except Exception as e:
    print(f"Error plotting training history: {e}")

## 4. Fine-tune the Model

Let's fine-tune the model by unfreezing some layers of the base model.

In [ ]:
try:
    # Unfreeze the top layers of the base model
    base_model = model.layers[0]
    base_model.trainable = True
    
    # Freeze all the layers except the last 30
    fine_tune_at = len(base_model.layers) - 30
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
        
    # Recompile the model with a lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Continue training with fine-tuning
    fine_tune_history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=callbacks
    )
    
    # Plot the fine-tuning history
    plot_training_history(fine_tune_history)
except Exception as e:
    print(f"Error during fine-tuning: {e}")

## 5. Save the Model

In [ ]:
# Save the trained model
model.save('ksl_recognition_model.h5')

# Save class names
import json
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

## 6. Convert to TensorFlow Lite

Now we'll convert our trained model to TensorFlow Lite format, which is optimized for mobile and edge devices.

In [ ]:
def convert_to_tflite(model, quantize=True):
    """
    Convert the trained model to TensorFlow Lite format
    """
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    if quantize:
        # Apply quantization for size reduction
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        # Apply full integer quantization
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
        
        # Generate representative dataset from training data
        def representative_dataset():
            for i in range(10):
                images, _ = next(iter(train_generator))
                yield [images]
        
        converter.representative_dataset = representative_dataset
    
    # Convert the model
    tflite_model = converter.convert()
    
    return tflite_model

In [ ]:
try:
    # Convert to TFLite without quantization
    tflite_model = convert_to_tflite(model, quantize=False)
    
    # Save the TFLite model
    with open('ksl_recognition_model.tflite', 'wb') as f:
        f.write(tflite_model)
    
    print("TFLite model size:", round(len(tflite_model) / (1024 * 1024), 2), "MB")
    
    # Try quantized version if the unquantized one works
    try:
        tflite_model_quant = convert_to_tflite(model, quantize=True)
        with open('ksl_recognition_model_quantized.tflite', 'wb') as f:
            f.write(tflite_model_quant)
        print("Quantized TFLite model size:", round(len(tflite_model_quant) / (1024 * 1024), 2), "MB")
    except Exception as e:
        print(f"Quantization failed: {e}. Using non-quantized model.")
except Exception as e:
    print(f"Error converting to TFLite: {e}")

## 7. Test the TFLite Model

Let's verify our TFLite model works correctly by testing it on a few sample images.

In [ ]:
def test_tflite_model(tflite_model_path, test_image_path, class_names):
    """
    Test the TFLite model on a sample image
    """
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    
    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Load and preprocess the test image
    img = Image.open(test_image_path)
    img = img.resize((IMG_HEIGHT, IMG_WIDTH))
    img_array = np.array(img, dtype=np.float32)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], img_array)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output results
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get the predicted class
    predicted_class = np.argmax(output_data[0])
    confidence = output_data[0][predicted_class]
    
    # Display results
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"Prediction: {class_names[predicted_class]} (Confidence: {confidence:.2f})")
    plt.axis('off')
    plt.show()
    
    return class_names[predicted_class], confidence

In [ ]:
# To test the TFLite model, uncomment and run this code with a test image path
'''
test_image_path = "path/to/test/image.jpg"  # Replace with actual test image path
tflite_model_path = "ksl_recognition_model.tflite"

# Load class names
with open('class_names.json', 'r') as f:
    class_names = json.load(f)
    
# Test the model
predicted_class, confidence = test_tflite_model(tflite_model_path, test_image_path, class_names)
print(f"Predicted sign: {predicted_class} with {confidence:.2%} confidence")
'''

## 8. Create a Simple Inference Script

This script can be used to recognize Kenyan Sign Language in real-time using a webcam.

In [18]:
def create_inference_script():
    """
    Create a Python script for real-time KSL recognition
    """
    script = '''
import cv2
import numpy as np
import tensorflow as tf
import json
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="ksl_recognition_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load class names
with open("class_names.json", "r") as f:
    class_names = json.load(f)

# Image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Open a video capture
cap = cv2.VideoCapture(0)

print("Press 'q' to quit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Prepare the image for inference
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], img_array)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output results
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get the predicted class
    predicted_class = np.argmax(output_data[0])
    confidence = output_data[0][predicted_class]
    
    # Display the result on the frame
    prediction_text = f"{class_names[predicted_class]}: {confidence:.2f}"
    cv2.putText(frame, prediction_text, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow('Kenyan Sign Language Recognition', frame)
    
    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
'''
    
    # Save the script
    with open('ksl_recognition_app.py', 'w') as f:
        f.write(script)
    
    print("Inference script saved as 'ksl_recognition_app.py'")

In [ ]:
# Create the inference script
create_inference_script()

## 9. Export the Model for Use in Mobile Applications

We can also prepare our model for use in mobile applications.

In [ ]:
# Create a zip file with the TFLite model and class names
import shutil

try:
    output_dir = 'ksl_recognition_model'
    os.makedirs(output_dir, exist_ok=True)
    
    # Copy the files to the output directory
    shutil.copy('ksl_recognition_model.tflite', f"{output_dir}/")
    shutil.copy('class_names.json', f"{output_dir}/")
    
    # Create a README file
    readme_content = """
# Kenyan Sign Language Recognition Model

This package contains a TensorFlow Lite model for recognizing Kenyan Sign Language (KSL) and translating it to English.

## Files:
- ksl_recognition_model.tflite: The trained TFLite model
- class_names.json: JSON file with the mapping of class indices to sign meanings

## Model Information:
- Input shape: 224x224x3 (RGB image normalized to [0,1])
- Output: Class probabilities corresponding to different KSL signs

## Usage:
1. Load the model using TensorFlow Lite
2. Preprocess the input image to 224x224 RGB
3. Normalize the pixel values to [0,1]
4. Run inference
5. Get the class with the highest probability
6. Map the class index to the sign meaning using class_names.json
"""
    
    with open(f"{output_dir}/README.md", 'w') as f:
        f.write(readme_content)
    
    # Create a zip file
    shutil.make_archive('ksl_recognition_model', 'zip', output_dir)
    
    print("Model package created: ksl_recognition_model.zip")
except Exception as e:
    print(f"Error creating model package: {e}")

## 10. Conclusion

We have successfully created a TensorFlow Lite model for Kenyan Sign Language recognition. The model can be used in various applications such as mobile apps, web applications, or embedded systems to help bridge the communication gap between Kenyan Sign Language users and English speakers.

### Next Steps:

1. **Improve the model**: Collect more data, try different architectures, or use more advanced techniques like attention mechanisms.
2. **Develop a complete mobile application**: Create Android/iOS apps that use the TFLite model for real-time sign language translation.
3. **Add sentence construction**: Extend the model to recognize sequences of signs and construct meaningful English sentences.
4. **Enable two-way translation**: Add functionality to translate English text back to Kenyan Sign Language animations.